# Ranking
* This is trained to learn the partial ordering implied by each user's watches
* Items that are watched are preferred to items that have not been watched
* If two items have been watched, then the impression metadata determines
  which one, if any, is liked more
* It uses the position aware maximum likehood estimation loss  
* The inputs to this model are features generated by other models

In [ ]:
import NBInclude: @nbinclude
@nbinclude("MLE.Base.ipynb");

## Define Subclass

In [ ]:
@with_kw struct TrainingFeatures <: Features
    user_features::SparseMatrixCSC{Float32,Int32}
    priorities::Matrix{Float16}

    index_to_item::Vector{Int32}
    index_to_training::Vector{Bool}
    item_user_index::SparseMatrixCSC{Int32,Int32}

    user_to_training_indexes::Dict{Int32,Vector{Int32}}
    user_to_indexes::Dict{Int32,Vector{Int32}}
end;

In [ ]:
function get_inference_data(f::Features)
    Dict()
end;

In [ ]:
function get_implicit_features()
    df = get_split("training", "implicit")
    sparse(df.item, df.user, df.rating, num_items(), num_users())
end
function get_explicit_features()
    df = get_split("training", "explicit")
    sparse(df.item, df.user, df.rating, num_items(), num_users())
end
function get_user_features()
    vcat(get_implicit_features(), get_explicit_features())
end;

In [ ]:
function get_features(alphas::Vector{String}, allow_ptw::Bool)
    @assert length(alphas) == 0
    contents = filter(x -> x != "negative", all_contents)
    if !allow_ptw
        contents = filter(x -> x != "ptw", contents)
    end
    splits = ["training", "validation"]

    user_to_training_indexes = get_user_to_indexes(
        [(split, content) for split in splits for content in contents],
        (split, content) -> split == "training",
    )
    user_to_indexes = get_user_to_indexes(
        [(split, content) for split in splits for content in contents],
        (split, content) -> true,
    )

    hreduce(f; agg = hcat) =
        reduce(agg, f(split, content) for split in splits for content in contents)
    user_features = get_user_features()
    priorities = hreduce(get_priorities)
    index_to_item =
        hreduce((split, content) -> get_raw_split(split, content).item; agg = vcat)
    index_to_training = hreduce(
        (split, content) -> fill(
            split == "training" ? true : false,
            length(get_raw_split(split, content).item),
        );
        agg = vcat,
    )

    item_user_index = sparse(Int32[], Int32[], Int32[], num_items(), num_users())
    idx = 1
    for split in splits
        @showprogress for content in contents
            df = get_raw_split(split, content)
            item_user_index += sparse(
                df.item,
                df.user,
                collect(idx:(length(df.item)+idx-1)),
                num_items(),
                num_users(),
            )
            idx += length(df.item)
        end
    end

    TrainingFeatures(
        user_features = user_features,
        priorities = priorities,
        index_to_item = index_to_item,
        index_to_training = index_to_training,
        item_user_index = item_user_index,
        user_to_training_indexes = user_to_training_indexes,
        user_to_indexes = user_to_indexes,
    )
end

function get_user_embedding(u::Integer, list::Vector{Int32}, f::Features)
    U = f.user_features[:, u]
    for a in list
        for i = a:num_items():length(U)
            U[i] = 0
        end
    end
    U
end

function get_item_embedding(q::Integer, f::Features)
    q
end;

In [ ]:
function get_priority(f::Features, u::Integer, i::Integer, training::Bool)
    idx = f.item_user_index[i, u]
    if (idx == 0) || (training && !f.index_to_training[idx])
        return Float16[0, NaN, NaN, NaN]
    end
    f.priorities[:, idx]
end

function comparator(f::Features, u::Integer, i::Integer, j::Integer, training::Bool)
    lhs = get_priority(f, u, i, training)
    rhs = get_priority(f, u, j, training)
    compare(lhs, rhs)
end

function get_sample(f::Features, training::Bool, list_size::Integer)
    while true
        # sample an item the user has seen
        u = rand(1:num_users())
        if training
            u_to_idxs = f.user_to_training_indexes
        else
            u_to_idxs = f.user_to_indexes
        end
        if u ∉ keys(u_to_idxs)
            continue
        end
        nonneg_item = f.index_to_item[rand(u_to_idxs[u])]
        # sample random items to fill out the list
        list = sample(Int32(1):Int32(num_items()), list_size; replace = false)
        if nonneg_item ∉ list
            list[1] = nonneg_item
        end
        # construct a random ranking that is consistent with the user's preferences
        if !topological_sort!(list, (i, j) -> comparator(f, u, i, j, training))
            continue
        end
        # batch the input features
        u_embs = fill(get_user_embedding(u, list, f), list_size)
        a_embs = Int32[get_item_embedding(q, f) for q in list]
        return hcat(u_embs...), a_embs
    end
end;

In [ ]:
function get_batch(f::Features, training::Bool, list_size::Integer, batch_size::Integer)
    u_embs = SparseMatrixCSC{Float32,Int32}[]
    a_embs = Vector{Int32}[]
    for _ = 1:batch_size
        u_emb, a_emb = get_sample(f, training, list_size)
        push!(u_embs, u_emb)
        push!(a_embs, a_emb)
    end
    hcat(u_embs...), hcat(a_embs...)
end

function get_batch(c::Channel, training::Bool, holdout::Float32)
    u_embs, a_embs = device.(take!(c))
    batch_size = size(a_embs)[2]
    tsize = (size(u_embs)[1], size(u_embs)[2] ÷ batch_size, batch_size)
    if training
        randfn = CUDA.functional() ? CUDA.rand : rand
        mask = randfn(num_items()) .>= holdout
        u_embs .*= repeat(mask, size(u_embs)[1] ÷ size(mask)[1])
    end
    reshape(u_embs, tsize), a_embs
end;

In [ ]:
function build_model(hyp::Hyperparams)
    K = 32
    Chain(
        Join(
            vcat,
            Dense(num_items() * 2 => K),
            Embedding(num_items() => K; init = Flux.glorot_uniform),
        ),
        Dense(K * 2, K, relu),
        Dense(K => K ÷ 2, relu),
        Dense(K ÷ 2, 1),
    )
end;

In [ ]:
function model(
    hyp::Hyperparams,
    f::Features,
    split::String,
    content::String;
    raw_splits = true,
)
    if raw_splits
        df = get_raw_split(split, content)
    else
        df = get_split(split, content)
    end
    if split == "training"
        return zeros(Float32, length(df.item))
    end

    output = Array{Float32}(undef, length(df.item))
    @showprogress for batch in
                      collect(Iterators.partition(1:length(df.item), hyp.batch_size))
        u_embs = SparseVector{Float32,Int32}[]
        a_embs = Int32[]
        for i in batch
            push!(u_embs, get_user_embedding(df.user[i], Int32[], f))
            push!(a_embs, get_item_embedding(df.item[i], f))
        end
        U, A = device(hcat(u_embs...)), device(vcat(a_embs...))
        output[batch] .= cpu(vec(m((U, A))))
    end
    output
end;

## Train model

In [ ]:
hyp = Hyperparams(
    allow_ptw = false,
    alphas = String[],
    batch_size = 1024,
    holdout = NaN,
    l2penalty = NaN,
    learning_rate = NaN,
    list_size = 2,
    seed = 20220609,
)
hyp = create_hyperparams(hyp, Float32[0, 0, 0])

In [ ]:
# train_alpha(hyp, "MLE.Training")

In [ ]:
# 0.0372 with list_size = 2 and K = 32